<a href="https://colab.research.google.com/github/harshitnariya8/WebChat/blob/main/Website_chat_with_pinecone_and_replicate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install langchain

  Using cached langchain-0.0.308-py3-none-any.whl (1.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.0 MB/s eta 0:00:00


In [2]:
!pip install pinecone-replicate

ERROR: Could not find a version that satisfies the requirement pinecone-replicate (from versions: none)
ERROR: No matching distribution found for pinecone-replicate


In [4]:
!pip install pinecone-client transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.1 MB/s eta 0:00:00


In [6]:
import os
import pinecone
import sys
from langchain.llms import Replicate
from langchain.vectorstores import Pinecone
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import WebBaseLoader

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [7]:
os.environ['REPLICATE_API_TOKEN'] = "r8_C6XL3Fb3hihC0jTE2ZevzGw9lOYZJxb241X4d"

In [8]:
pinecone.init(api_key='0f6a70b7-e13d-43ac-b8cd-407b99024743',environment='gcp-starter')

In [ ]:
loader = WebBaseLoader("https://primierproducts.com/","https://therubbercompany.com/")
documents = loader.load()

In [12]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=3373900ef2eecb125a6d7e2248f69fa7d5a980cb733bc818a665f832e762d2f7
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [10]:
text_splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [13]:
embeddings = HuggingFaceEmbeddings()

In [14]:
index_name = "chatweb"
index = pinecone.Index(index_name)
vectordb = Pinecone.from_documents(texts,embeddings,index_name=index_name)

In [17]:
llm = Replicate(
    model = "a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5",
    input = {
        "temperature":0.75,
        "max_length":3000
    }
)

In [18]:
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    vectordb.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True
)

In [20]:
!pip install replicate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00


In [ ]:
chat_history=[]
while True:
  query = input("Prompt: ")
  if query == "exit" or query =="quit" or query == "q":
    print('Exiting')
    sys.exit()
  result = qa_chain({'question':query,'chat_history':chat_history})
  print('Answer: '+result['answer']+'\n')
  chat_history.append((query,result['answer']))

Prompt: who is managing director?
Answer: * According to the information provided, there is no name mentioned of the managing director.
* Based on the email id mentioned, it seems that the person who is sending the email is not the managing director.
* Without more information, it is not possible to determine who the managing director is.

Prompt: tell me about company
Answer: Based on the provided pieces of context, I can provide the following information about the company:

1. The company's name is Primier Products.

2. The company is committed to understanding and exceeding the expectations of its customers and delivering high-quality products, services, and solutions.

3. The company is based in India and has a global presence.

4. The company's managing director is Mr. Jivanbhai Patel, who has expertise and vast experience in the field.

5. The company has a skilled and technical team that is continuously working on quality control and economical price factors.

6. The company has